Pierwszy z nich to wyszukiwarka członków ruchu oporu. Możemy wyszukiwać ich z użyciem imienia podanego w formie mianownika, a w odpowiedzi otrzymamy listę miejsc, w których ich widziano.
https://centrala.ag3nts.org/people 

Drugi system to wyszukiwarka miejsc odwiedzonych przez konkretne osoby. Podajesz nazwę miasta do sprawdzenia (bez polskich znaków) i w odpowiedzi dowiadujesz się, których z członków ruchu oporu tam widziano.
https://centrala.ag3nts.org/places

Interfejs dla obu systemów wygląda tak samo. Przyjmuje on zapytanie w formacie JSON jak poniżej:

{
 "apikey":"TWÓJ KLUCZ",
 "query": "IMIE lub MIASTO"
}

Niestety, przy obu systemach ktoś majstrował i dane zwracane przez nie mogą być niekompletne. Musisz jednak poradzić sobie z tym, co masz.

Zdobyliśmy także część notatki na temat Barbary, pochodząca z systemu zbierającego informacje o osobach poszukiwanych.

https://centrala.ag3nts.org/dane/barbara.txt

Połącz wszystkie dane w jedną całość i uzupełnij brakujące informacje. Kto był współpracownikiem Aleksandra i Barbary? Z kim widział się Rafał? Być może znalezienie informacji na ten temat pozwoli nam wytypować kolejne miejsce, w którym warto poszukać Barbary.

Co należy zrobić w zadaniu?

Ściągnij notatkę (plik TXT) na temat Barbary.

Zastanów się jakie osoby i jakie nazwy miast są wspomniane w notatce

Odpytaj o wspomniane poszlaki odpowiednie API

Istnieje szansa, że z danych otrzymanych przez API otrzymasz kolejne imiona lub nazwy miast

Odpytuj o nie kolejno tak długo, aż znajdziesz miasto, w którym znajduje się BARBARA

Istnieje szansa, że idąc jakąś pokrętną drogą natrafisz na sekretną flagę 🚩

Gdy namierzysz miasto, w którym znajduje się BARBARA, wyślij nazwę miasta do centrali (/report) do zadania loop. Nazwa miasta ma być w tym samym formacie jak została zwrócona z API czyli np. LODZ. Wysyłka nazwy Łódź nie zostanie zaliczona. 

Podpowiedź: tego zadania nie da się rozwiązać z użyciem, zaledwie jednego prompta. Wymagane jest pewne zapętlenie zapytań, ale uważaj, aby nie zapętlić się w nieskończoność. Pamiętaj, że API odpytujemy słowami w mianowniku i bez polskich znaków (SLASK, a nie ŚLĄSKIEGO i GRZESIEK, a nie GRZEŚKOWI).

In [261]:
import os
import requests
import json
from dotenv import load_dotenv

In [262]:
load_dotenv()

personal_api_key = os.getenv("PERSONAL_API_KEY")

In [263]:
def list_json_keys(json_data, prefix=""):
    """Recursively list all keys in a JSON structure"""
    keys = []
    if isinstance(json_data, dict):
        for key, value in json_data.items():
            current_key = f"{prefix}.{key}" if prefix else key
            keys.append(current_key)
            if isinstance(value, (dict, list)):
                keys.extend(list_json_keys(value, current_key))
    elif isinstance(json_data, list):
        for i, item in enumerate(json_data):
            current_key = f"{prefix}[{i}]"
            if isinstance(item, (dict, list)):
                keys.extend(list_json_keys(item, current_key))
    return keys

In [264]:
def save_information_about_people(information_about_people):
    with open("information_about_people.json", "w") as file:
        json.dump(information_about_people, file, indent=4)

In [281]:
anti_spam_list = ['no data', '"QUERY"', 'Ł', '[', ']', '**', 'query', 'should', '[**RESTRICTED DATA**]']


In [282]:
def load_information_about_people(list_of_people) :
    information_about_people = []
    if not os.path.exists("information_about_people.json"):
        list_of_people = ['ALEKSANDER', 'BARBARA', 'ADAM', 'RAFAL', 'AZAZEL']
    else:
        with open("information_about_people.json", "r") as file:    
            data = json.load(file)
            for person in data:
                # print(person)
                for key, value in person.items():
                    list_of_people.append(key)
    for person in list_of_people:
        if person not in anti_spam_list:   
            json_query_people = {
            "apikey":personal_api_key,
            "query": person
        }  
            response = requests.post(url="https://centrala.ag3nts.org/people", json=json_query_people)
            information_about_people.append({f"{person}" : response.json()["message"]})
    # print(json.dumps(information_about_people, indent=4))
    return information_about_people

In [300]:
def check_city(city):
    answer = city
    answer_json = {
        "task": "loop",
        "apikey": personal_api_key,
        "answer": answer
    }
    answer_url = "https://centrala.ag3nts.org/report"
    answer_response = requests.post(answer_url, json=answer_json)
    print(answer_response.request.body)  # Displays the request body

    # Assuming `login_response` is the response object from the login request
    print("Status Code:", answer_response.status_code)  # Displays the status code (e.g., 200)
    print("Response Body:", answer_response.text.encode('utf-8'))       # Displays the response content as a string    

In [301]:
def use_information_about_people(list_of_people, information_about_people): 
    information_about_people_in_cities = []
    list_of_cities = []
    for information in information_about_people:
        if information not in anti_spam_list:
            cities = list(information.values())[0].split(" ")
            if cities not in anti_spam_list:
                for city in cities:
                    if city not in list_of_cities:
                        list_of_cities.append(city)
    print(list_of_cities)
    for city in list_of_cities:
        json_query_city = {
            "apikey":personal_api_key,
            "query": city
        }
        response = requests.post(url="https://centrala.ag3nts.org/places", json=json_query_city)
        response_json = json.loads(response.text)
        print(response_json["message"])      
        information_about_people_in_cities.append({f"{city}": response_json["message"]})     
        if response_json.get("message") and response_json["message"] not in anti_spam_list:
            people_in_city = response_json["message"]
            if people_in_city not in anti_spam_list and people_in_city.isupper():
                for person in people_in_city.split(" "):
                    if person not in list_of_people:
                        list_of_people.append(person)
    print(f"list_of_people use information about people: {list_of_people}")
    with open("information_about_people_in_cities.json", "w") as file:
        json.dump(information_about_people_in_cities, file, indent=4)
    return list_of_people


In [302]:
list_of_people = []
if os.path.exists("information_about_people.json"):
    os.remove("information_about_people.json")
for i in range(5):
    information_about_people = load_information_about_people(list_of_people)
    list_of_people = use_information_about_people(list_of_people, information_about_people)
    save_information_about_people(information_about_people)

with open("information_about_people_in_cities.json", "r") as file:
    data = json.load(file)
    for information in data:
        for key, value in information.items():
            print(key)
            print(value)
            if value == "BARBARA":
                check_city(key)
                break

['KRAKOW', 'LUBLIN', 'WARSZAWA', '[**RESTRICTED', 'DATA**]', 'CIECHOCINEK', 'GRUDZIADZ', 'ELBLAG']
ALEKSANDER BARBARA ADAM
[**RESTRICTED DATA**]
RAFAŁ ALEKSANDER ANDRZEJ
query should contain only capital letters
query should contain only capital letters
ADAM GABRIEL
RAFAL AZAZEL
BARBARA
list_of_people use information about people: ['ALEKSANDER', 'BARBARA', 'ADAM', 'RAFAŁ', 'ANDRZEJ', 'GABRIEL', 'RAFAL', 'AZAZEL']
['KRAKOW', 'LUBLIN', 'WARSZAWA', '[**RESTRICTED', 'DATA**]', 'CIECHOCINEK', 'query', 'should', 'contain', 'only', 'capital', 'letters', 'GRUDZIADZ', 'FROMBORK', 'ELBLAG']
ALEKSANDER BARBARA ADAM
[**RESTRICTED DATA**]
RAFAŁ ALEKSANDER ANDRZEJ
query should contain only capital letters
query should contain only capital letters
ADAM GABRIEL
no data found for "QUERY" in module "places"
no data found for "SHOULD" in module "places"
no data found for "CONTAIN" in module "places"
no data found for "ONLY" in module "places"
no data found for "CAPITAL" in module "places"
no data found f